In [2]:
from google.colab import files
uploaded =files.upload()

Saving shakespeare.txt to shakespeare.txt


In [0]:
import numpy as np

In [0]:
# reading text file
document = open("shakespeare.txt").read()

In [6]:

t_char=set(document)
len(t_char)
print("total number of distinct characters in documnet is  : ",len(t_char))

total number of distinct characters in documnet is  :  67


In [7]:
# converting documnet to lowercase characters
doc = document.lower()
char = set(doc)
print("total number of character after converting character into lowercase letters is:",len(char))

total number of character after converting character into lowercase letters is: 41


In [8]:
l_char=sorted(list(char))
l_char

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [0]:
#converting character to integer, making dictionary of characters
char_to_int={}
for i,j in enumerate(l_char):
    char_to_int[j]=i

In [10]:
len(char_to_int)

41

In [11]:
n_char=len(doc)
print("total numnebr of characters in document is:",n_char)
n_dist_char = len(l_char)
print("total number of distinct character in document:", n_dist_char)

total numnebr of characters in document is: 4573338
total number of distinct character in document: 41


In [0]:
# converting text into 2 dimensional array
steps= 125
x=[]
y=[]
for i in range (100000-steps):
  inp = doc[i:i+steps]
  out = doc[i+steps]
  lst=[]
  for i in range(steps):
    lst.append(char_to_int[inp[i]])
  x.append(lst)
  y.append(char_to_int[out])

    
    
    

In [13]:
len_x = len(x)
print("lenth of feedable data to RNN:", len(x))
print("step_size:",steps)

lenth of feedable data to RNN: 99875
step_size: 125


In [14]:
import tensorflow as tf
X=tf.one_hot(x,len(char_to_int))
print(type(X))



<class 'tensorflow.python.framework.ops.Tensor'>


In [15]:
print(X)

Tensor("one_hot:0", shape=(99875, 125, 41), dtype=float32)


In [0]:
x= np.array(x)
train_X = (np.arange(x.max()+1) == x[...,None]-1).astype(int)

In [17]:
print(train_X.shape)

(99875, 125, 41)


In [18]:

y = np.array(y)
print(type(y))
print(y.shape)
y_one_hot = np.eye(41)[y]
print(y_one_hot.shape)

<class 'numpy.ndarray'>
(99875,)
(99875, 41)


In [0]:
y_onehot= tf.one_hot(y,41)

In [20]:
print(y_onehot)
steps= 125
n_neurons = 150
n_outputs = len(char)

Tensor("one_hot_1:0", shape=(99875, 41), dtype=float32)


In [0]:
import tensorflow as tf
X = tf.placeholder(dtype=tf.float32, shape=[None, steps, 41],name = "X")
Y = tf.placeholder(tf.float32, [None,41], name= "Y")



In [22]:
# defining rnn cell
from tensorflow.contrib.layers import fully_connected
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
#print(outputs)
#rint(states)


W0704 08:41:18.152283 140045698836352 deprecation.py:323] From <ipython-input-22-e85f20bda27d>:3: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0704 08:41:18.154759 140045698836352 deprecation.py:323] From <ipython-input-22-e85f20bda27d>:4: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0704 08:41:18.382816 140045698836352 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argumen

In [23]:
print(outputs)
print(states)

Tensor("rnn/transpose_1:0", shape=(?, 125, 256), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(?, 256), dtype=float32)


In [24]:
logits = fully_connected(states, n_outputs, activation_fn=None)
print(logits)



Tensor("fully_connected/BiasAdd:0", shape=(?, 41), dtype=float32)


In [25]:
xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits)

W0704 08:41:18.915885 140045698836352 deprecation.py:323] From <ipython-input-25-1ed2ec414530>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# loss minimization
learning_rate=0.01
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
#correct = tf.nn.in_top_k(logits, Y, 1)
#accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()

In [0]:
batch_size = 128


In [29]:
m  =int(len_x/128)
m

780

**Building** **Model**

In [43]:
#building the RNN model
epoch  = 10
loss_value = 99999
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(init)
    for i in range(epoch):
      for batch in range(m):
          x_batch = train_X[batch*batch_size:min((batch+1)*batch_size, len_x)]
          y_batch = y_one_hot[batch*batch_size:min((batch+1)*batch_size, len_x)]
          _,los_cal = sess.run([training_op,loss],feed_dict={X:x_batch,Y:y_batch})
          
          
      if los_cal<loss_value:
          saver.save(sess,"result")
          loss_value = los_cal
      print("epoch " + str(i+1) + ", Loss= " + "{:.6f}".format(los_cal) )   
    
    
    
    

epoch 1, Loss= 2.974978
epoch 2, Loss= 2.711518
epoch 3, Loss= 2.678522
epoch 4, Loss= 2.705904
epoch 5, Loss= 2.669420
epoch 6, Loss= 2.669443
epoch 7, Loss= 2.688489
epoch 8, Loss= 2.698829
epoch 9, Loss= 2.670262
epoch 10, Loss= 2.869717


**Generating the text using RNN Model**

In [162]:
a = np.random.randint(1,100000)

a

61363

In [0]:
x_test = train_X[a]


In [0]:
lst1 = list(x_test)
str= ""

In [0]:
def softmax(inputs):
    return np.exp(inputs) / float(sum(np.exp(inputs)))

In [0]:
int_to_char={}
for i,j in enumerate(l_char):
    int_to_char[i]=j
output =[]


**Running the code using pretrained weight of the model, which has minimum loss**

In [0]:
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess,"result")
for i in range(500):
  x_test_feed = x_test.reshape(1,125,41)
  logit = sess.run(logits,feed_dict={X:x_test_feed})
  output_softmax =softmax(logit[0])
  ch = np.argmax(output_softmax)
  output.append(ch)
  str=str + (int_to_char[ch])
  new_ch = np.eye(41)[ch]
  lst1.append(new_ch)
  x_test = np.array(lst1)
  x_test = x_test[i+1:]
#print(str)
#print(output)
  
  
  
  

In [160]:
print(output)

[19, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1, 0, 34, 32, 1

In [161]:
import sys
for i in range(500):
  sys.stdout.write(int_to_char[output[i]])

e 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
tr 
t